In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import requests as req
import pandas as pd

# Web Scrapping (ESPN)

In [2]:
url = 'https://www.espn.com/soccer/standings/_/league/ESP.1/season/2018'

In [3]:
html=req.get(url).content

In [4]:
soup=bs(html, 'html.parser')

In [5]:
table_html = soup.find_all('div', class_="flex")

In [6]:
index = []

for e in table_html:
    if e.tr:
        index.append(e.tr.text)

In [7]:
index  # Realmente extraer esto no nos aporta demasiado, pero nos permite comprender mejor la estructura html de esta url.

['2018/2019']

In [8]:
# El nombre de la primera columna (index) tiene diferente etiquetado html que el resto de nombres de columnas (tienen un div de otra clase).
# Tenemos que acceder al html del resto de las columnas ('div', class_="Table__Scroller").

table_html_2 = soup.find_all('div', class_="Table__Scroller")

In [9]:
# Ya tenemos el div completo. Ahra tenemos que acceder al nombre de las columnas (texto de <a> dentro del <div>).

col_html = []
for e in table_html_2:
    col_html.append(e.find_all('a'))
       

In [10]:
col_html  # Ya tenemos los diferentes tags <a>.

[[<a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/gamesplayed/dir/desc/season/2018" tabindex="0">GP</a>,
  <a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/wins/dir/desc/season/2018" tabindex="0">W</a>,
  <a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/ties/dir/desc/season/2018" tabindex="0">D</a>,
  <a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/losses/dir/asc/season/2018" tabindex="0">L</a>,
  <a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/pointsfor/dir/desc/season/2018" tabindex="0">F</a>,
  <a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/pointsagainst/dir/asc/season/2018" tabindex="0">A</a>,
  <a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/pointdifferential/dir/desc/season/2018" tabindex="0">GD</a>,
  <a class="AnchorLink" href="/soccer/standings/_/league/ESP.1/sort/points/dir/desc/season/2018" tabindex="0">P</a>]]

In [11]:
# Accedemos al texto (doble bucle porque 'row_html' es una lista dentro de otra lista (hicimos find_all)).

col_names=[]
for e in col_html:
    for i in e:
        col_names.append(i.text)

In [12]:
col_names # Ya tenemos los nombres de las columnas (primer paso para ir construyendo nuestro DataFrame)

['GP', 'W', 'D', 'L', 'F', 'A', 'GD', 'P']

Ahora que ya tenemos el nombre de las columnas, seguimos extrayendo los datos.

Como la primera columna (equipos) está estructurada de manera independiente a las otras, debemos hacerlo por separado (primero extraeremos los equipos y luego los valores de la tabla).

In [13]:
# Empezamos extrayendo los nombres de los equipos.

teams_html = soup.find_all('span', class_="hide-mobile")

In [14]:
teams = []
for e in teams_html:
    teams.append(e.text)

In [15]:
teams # Ya tenemos los nombres de los equipos (primera columna)

['Barcelona',
 'Atletico Madrid',
 'Real Madrid',
 'Valencia',
 'Getafe',
 'Sevilla',
 'Espanyol',
 'Athletic Club',
 'Real Sociedad',
 'Real Betis',
 'Alavés',
 'Eibar',
 'Leganés',
 'Villarreal',
 'Levante',
 'Real Valladolid',
 'Celta Vigo',
 'Girona',
 'Huesca',
 'Rayo Vallecano']

Ahora que tenemos los nombres de las columnas y los equipos, extraemos los datos de la tabla.

In [16]:
data_html = soup.find_all('tbody', class_= "Table__TBODY")

In [17]:
len(data_html)

2

In [18]:
# Seleccionamos los 'tr' en el segundo elemento de data_html (es el 'tbody' de la clase "Table__TBODY" que nos interesa).

data_html2 = data_html[1].find_all('tr')

In [19]:
# Primero hacemos la prueba para un elemento de data_html2.

data_html2[1]

<tr class="filled Table__TR Table__TR--sm Table__even" data-idx="1"><td class="Table__TD"><span class="stat-cell">38</span></td><td class="Table__TD"><span class="stat-cell">22</span></td><td class="Table__TD"><span class="stat-cell">10</span></td><td class="Table__TD"><span class="stat-cell">6</span></td><td class="Table__TD"><span class="stat-cell">55</span></td><td class="Table__TD"><span class="stat-cell">29</span></td><td class="Table__TD"><span class="stat-cell clr-positive">+26</span></td><td class="Table__TD"><span class="stat-cell">76</span></td></tr>

In [20]:
data_html2[1].find_all('td')

[<td class="Table__TD"><span class="stat-cell">38</span></td>,
 <td class="Table__TD"><span class="stat-cell">22</span></td>,
 <td class="Table__TD"><span class="stat-cell">10</span></td>,
 <td class="Table__TD"><span class="stat-cell">6</span></td>,
 <td class="Table__TD"><span class="stat-cell">55</span></td>,
 <td class="Table__TD"><span class="stat-cell">29</span></td>,
 <td class="Table__TD"><span class="stat-cell clr-positive">+26</span></td>,
 <td class="Table__TD"><span class="stat-cell">76</span></td>]

In [21]:
[e.text for e in data_html2[1].find_all('td')]  

# Comprobamos que funciona, así que aplicamos el proceso anterior a todo el contenido de la tabla.

['38', '22', '10', '6', '55', '29', '+26', '76']

In [22]:
# Primero creamos una lista con todos los 'td' en todos los elemento del 'tbody'.

data_prov = []
for e in data_html2:
    data_prov.append(e.find_all('td'))

In [23]:
# Aplicamos el doble bucle sobre la lista previamente creada.

# El doble bucle con la lista 'prov' nos permite añadir a data_def listas con los valores de cada fila.
# Es decir, data_def será una lista de listas (podemos meterlo como data en un DataFrame).
#Si no hacemos esto, lo que tendríamos es una única lista con todos los valores y esto no es lo que queremos.

data_def = []

for e in data_prov:
    prov = []
    
    for i in e:
        prov.append(i.text)
        
    data_def.append(prov)

In [24]:
# Observamos nuestro data (cada elemento de la lista, es una lista que contiene los valores para una fila).

data_def

[['38', '26', '9', '3', '90', '36', '+54', '87'],
 ['38', '22', '10', '6', '55', '29', '+26', '76'],
 ['38', '21', '5', '12', '63', '46', '+17', '68'],
 ['38', '15', '16', '7', '51', '35', '+16', '61'],
 ['38', '15', '14', '9', '48', '35', '+13', '59'],
 ['38', '17', '8', '13', '62', '47', '+15', '59'],
 ['38', '14', '11', '13', '48', '50', '-2', '53'],
 ['38', '13', '14', '11', '41', '45', '-4', '53'],
 ['38', '13', '11', '14', '45', '46', '-1', '50'],
 ['38', '14', '8', '16', '44', '52', '-8', '50'],
 ['38', '13', '11', '14', '39', '50', '-11', '50'],
 ['38', '11', '14', '13', '46', '50', '-4', '47'],
 ['38', '11', '12', '15', '37', '43', '-6', '45'],
 ['38', '10', '14', '14', '49', '52', '-3', '44'],
 ['38', '11', '11', '16', '59', '66', '-7', '44'],
 ['38', '10', '11', '17', '32', '51', '-19', '41'],
 ['38', '10', '11', '17', '53', '62', '-9', '41'],
 ['38', '9', '10', '19', '37', '53', '-16', '37'],
 ['38', '7', '12', '19', '43', '65', '-22', '33'],
 ['38', '8', '8', '22', '41', '

Generamos el DataFrame a partir del scrapeo de ESPN.

In [25]:
col_names

['GP', 'W', 'D', 'L', 'F', 'A', 'GD', 'P']

In [26]:
teams # Será el index de nuestro DataFrame

['Barcelona',
 'Atletico Madrid',
 'Real Madrid',
 'Valencia',
 'Getafe',
 'Sevilla',
 'Espanyol',
 'Athletic Club',
 'Real Sociedad',
 'Real Betis',
 'Alavés',
 'Eibar',
 'Leganés',
 'Villarreal',
 'Levante',
 'Real Valladolid',
 'Celta Vigo',
 'Girona',
 'Huesca',
 'Rayo Vallecano']

In [27]:
len(col_names)==len(data_def[0])   # Podemos generar el DataFrame sin problemas.

True

In [28]:
len(teams)==len(data_def)     # Sí coincide así que no ha habido fallos (hay tantos equipos como registros de valores (filas)).

True

In [29]:
teams_stats = pd.DataFrame(data_def, columns = col_names)

In [30]:
teams_stats  # Tenemos el DataFrame con todos los valores ('data_def'). Ahora añadiremos los equipos y lo definiremos como index.

,GP,W,D,L,F,A,GD,P
0,38,26,9,3,90,36,+54,87
1,38,22,10,6,55,29,+26,76
2,38,21,5,12,63,46,+17,68
3,38,15,16,7,51,35,+16,61
4,38,15,14,9,48,35,+13,59
5,38,17,8,13,62,47,+15,59
6,38,14,11,13,48,50,-2,53
7,38,13,14,11,41,45,-4,53
8,38,13,11,14,45,46,-1,50
9,38,14,8,16,44,52,-8,50


In [31]:
# Añadimos la columna 'teams'.

teams_stats['Teams'] = teams

In [32]:
# Definimos 'Team' como index.

teams_stats.set_index('Teams',inplace = True)

In [33]:
# Ya tenemos nuestro DataFrame perfectamente estructurado.
# Debemos tener en cuenta que los valores recogidos en el DF son de tipo string.
# Esto ahora no supone ningún problema, pero es algo que debemos tener en cuenta si queremos trabajar con los datos más adelante.

teams_stats

,GP,W,D,L,F,A,GD,P
Teams,,,,,,,,
Barcelona,38,26,9,3,90,36,+54,87
Atletico Madrid,38,22,10,6,55,29,+26,76
Real Madrid,38,21,5,12,63,46,+17,68
Valencia,38,15,16,7,51,35,+16,61
Getafe,38,15,14,9,48,35,+13,59
Sevilla,38,17,8,13,62,47,+15,59
Espanyol,38,14,11,13,48,50,-2,53
Athletic Club,38,13,14,11,41,45,-4,53
Real Sociedad,38,13,11,14,45,46,-1,50


# Limpieza

Queremos que el equipo ('Team') actúe como key compartida en las relaciones con otras tablas, así que haremos que estos valores coincidan con el utilizado en las estadísticas de jugadores (archivo csv), sustituyendo los espacios por "_".

In [52]:
Team = ['FC_Barcelona', 'Atlético_de_Madrid', 'Real_Madrid',  'Valencia_CF',  'Getafe_CF', 'Sevilla_FC', 'RCD_Espanyol',
        'Athletic_Club', 'Real_Sociedad', 'Real_Betis', 'D._Alavés','SD_Eibar', 'CD_Leganés', 'Villarreal_CF',  
        'Levante_UD','R._Valladolid_CF', 'RC_Celta', 'Girona_FC', 'SD_Huesca','Rayo_Vallecano' ]

In [53]:
teams_stats['Team'] = Team   

In [55]:
teams_stats.set_index('Team',inplace = True)

In [56]:
teams_stats

,GP,W,D,L,F,A,GD,P
Team,,,,,,,,
FC_Barcelona,38,26,9,3,90,36,+54,87
Atlético_de_Madrid,38,22,10,6,55,29,+26,76
Real_Madrid,38,21,5,12,63,46,+17,68
Valencia_CF,38,15,16,7,51,35,+16,61
Getafe_CF,38,15,14,9,48,35,+13,59
Sevilla_FC,38,17,8,13,62,47,+15,59
RCD_Espanyol,38,14,11,13,48,50,-2,53
Athletic_Club,38,13,14,11,41,45,-4,53
Real_Sociedad,38,13,11,14,45,46,-1,50


Cambiamos el tipo de dato de todos los valores a int8 por si en un futuro queremos hacer cálculos con ellos.

In [71]:
teams_stats.GP.astype(dtype="int8")
teams_stats.W.astype(dtype="int8")
teams_stats.D.astype(dtype="int8")
teams_stats.L.astype(dtype="int8")
teams_stats.F.astype(dtype="int8")
teams_stats.A.astype(dtype="int8")
teams_stats.GD.astype(dtype="int8")
teams_stats.F.astype(dtype="int8")

Team
FC_Barcelona          90
Atlético_de_Madrid    55
Real_Madrid           63
Valencia_CF           51
Getafe_CF             48
Sevilla_FC            62
RCD_Espanyol          48
Athletic_Club         41
Real_Sociedad         45
Real_Betis            44
D._Alavés             39
SD_Eibar              46
CD_Leganés            37
Villarreal_CF         49
Levante_UD            59
R._Valladolid_CF      32
RC_Celta              53
Girona_FC             37
SD_Huesca             43
Rayo_Vallecano        41
Name: F, dtype: int8